# _The Technical Interview_

This notebook will provide a review of the following Python concepts: `enum` module

# _Data Structures_

### _Creating Enumerations_

In [2]:
import enum

class BugStatus(enum.Enum):
    
    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1
    
print('\nMember name: {}'.format(BugStatus.wont_fix.name))
print('Member value: {}'.format(BugStatus.wont_fix.value))


Member name: wont_fix
Member value: 4


### _Iteration_

In [3]:
class BugStatus(enum.Enum):
    
    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1
    
for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

new             = 7
incomplete      = 6
invalid         = 5
wont_fix        = 4
in_progress     = 3
fix_committed   = 2
fix_released    = 1


### _Comparing Enums_

- enumeration members __not__ ordered
- comparison only by identity and equality

In [4]:
class BugStatus(enum.Enum):
    
    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

In [5]:
actual_state = BugStatus.wont_fix
desired_state = BugStatus.fix_released

print('Equality:',
     actual_state == desired_state,
     actual_state == BugStatus.wont_fix)
print('Identity:',
     actual_state is desired_state,
     actual_state is BugStatus.wont_fix)
print('Ordered by value:')
try:
    print('\n'.join('  ' + s.name for s in sorted(BugStatus)))
except TypeError as err:
    print(' Cannot sort: {}'.format(err))

Equality: False True
Identity: False True
Ordered by value:
 Cannot sort: '<' not supported between instances of 'BugStatus' and 'BugStatus'


In [6]:
class BugStatus(enum.IntEnum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

In [7]:
print('Ordered by value:')
print('\n'.join('  ' + s.name for s in sorted(BugStatus)))

Ordered by value:
  fix_released
  fix_committed
  in_progress
  wont_fix
  invalid
  incomplete
  new


### _Unique Enumeration Values_

- enum members with same value are tracked as alias references to the same member object
- aliases do not cause repeated values to be present in iterator for `Enum`

In [8]:
class BugStatus(enum.Enum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

    by_design = 4
    closed = 1

In [9]:
for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

new             = 7
incomplete      = 6
invalid         = 5
wont_fix        = 4
in_progress     = 3
fix_committed   = 2
fix_released    = 1


In [11]:
print('\nSame: by_design is wont_fix: ',
     BugStatus.by_design is BugStatus.wont_fix)
print('Same: closed is fix_released: ',
     BugStatus.closed is BugStatus.fix_released)


Same: by_design is wont_fix:  True
Same: closed is fix_released:  True


In [12]:
# to require all members to have unique values, add the @unique decorator to the Enum

@enum.unique
class BugStatus(enum.Enum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

    # This will trigger an error with unique applied.
    by_design = 4
    closed = 1

ValueError: duplicate values found in <enum 'BugStatus'>: by_design -> wont_fix, closed -> fix_released

Members with repeated values trigger a `ValueError` exception when the `Enum` class is being interpreted.

### _Creating Enumerations Programmatically_

- sometimes more convenient to create enumerations programmatically, rather than hard-coding in class definition
- `Enum` supports passing the member names and values to the class constructor

In [14]:
BugStatus = enum.Enum(
    value = 'BugStatus',
    names = ('fix_released fix_committed in_progress '
            'wont_fix invalid incomplete new'),
)

print('Member: {}'.format(BugStatus.new))

print('\nAll members:')
for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

Member: BugStatus.new

All members:
fix_released    = 1
fix_committed   = 2
in_progress     = 3
wont_fix        = 4
invalid         = 5
incomplete      = 6
new             = 7


- `value` argument is name of the enumeration
- `names` argument lists the members of the enumeration
    - when single string is passed, it's split on whitespace and commas
    - resulting tokens are used as names for the members, which are automatically assigned values starting with 1
    
- For more control over values, `names` string can be replaced with a sequence of two-part tuples or a dictionary mapping names to values

In [15]:
BugStatus = enum.Enum(
    value='BugStatus',
    names=[
        ('new', 7),
        ('incomplete', 6),
        ('invalid', 5),
        ('wont_fix', 4),
        ('in_progress', 3),
        ('fix_committed', 2),
        ('fix_released', 1)
    ],
)

print('All members:')
for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

All members:
new             = 7
incomplete      = 6
invalid         = 5
wont_fix        = 4
in_progress     = 3
fix_committed   = 2
fix_released    = 1


### _Non-integer Member Values_

- Enum member values are not restricted to integers
- Any type of object can be associated with a member
- if value is a tuple, members are passed as individual arguments to `__init__()`
- in this example, each member value is tuple containing numerical ID and list of valid transitions away from current state

In [18]:
class BugStatus(enum.Enum):

    new = (7, ['incomplete',
               'invalid',
               'wont_fix',
               'in_progress'])
    incomplete = (6, ['new', 'wont_fix'])
    invalid = (5, ['new'])
    wont_fix = (4, ['new'])
    in_progress = (3, ['new', 'fix_committed'])
    fix_committed = (2, ['in_progress', 'fix_released'])
    fix_released = (1, ['new'])

    def __init__(self, num, transitions):
        self.num = num
        self.transitions = transitions

    def can_transition(self, new_state):
        return new_state.name in self.transitions


print('Name:', BugStatus.in_progress)
print('Value:', BugStatus.in_progress.value)
print('Custom attribute:', BugStatus.in_progress.transitions)
print('Using attribute:',
      BugStatus.in_progress.can_transition(BugStatus.new))

Name: BugStatus.in_progress
Value: (3, ['new', 'fix_committed'])
Custom attribute: ['new', 'fix_committed']
Using attribute: True


- For more complex cases, tuples might be unwieldy
- Member values can be any type of object
- dictionaries can be used for cases where a lot separate attributes to track for each enum value
- complex values are passed directly to `__init__()` as the only argument other than `self`

In [19]:
class BugStatus(enum.Enum):

    new = {
        'num': 7,
        'transitions': [
            'incomplete',
            'invalid',
            'wont_fix',
            'in_progress',
        ],
    }
    incomplete = {
        'num': 6,
        'transitions': ['new', 'wont_fix'],
    }
    invalid = {
        'num': 5,
        'transitions': ['new'],
    }
    wont_fix = {
        'num': 4,
        'transitions': ['new'],
    }
    in_progress = {
        'num': 3,
        'transitions': ['new', 'fix_committed'],
    }
    fix_committed = {
        'num': 2,
        'transitions': ['in_progress', 'fix_released'],
    }
    fix_released = {
        'num': 1,
        'transitions': ['new'],
    }

    def __init__(self, vals):
        self.num = vals['num']
        self.transitions = vals['transitions']

    def can_transition(self, new_state):
        return new_state.name in self.transitions


print('Name:', BugStatus.in_progress)
print('Value:', BugStatus.in_progress.value)
print('Custom attribute:', BugStatus.in_progress.transitions)
print('Using attribute:',
      BugStatus.in_progress.can_transition(BugStatus.new))

Name: BugStatus.in_progress
Value: {'num': 3, 'transitions': ['new', 'fix_committed']}
Custom attribute: ['new', 'fix_committed']
Using attribute: True
